### Non-functioning notebook used to share code

In [ ]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
spacex_df['Booster Version Short'] = spacex_df['Booster Version'].str.split().str[1]
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                dcc.Dropdown(id='site-dropdown',options=[
                                        {'label':'All Sites','value':'all'},
                                        {'label':'CCAFS LC-40','value':'CCAFS LC-40'},
                                        {'label':'VAFB SLC-4E','value':'VAFB SLC-4E'},
                                        {'label':'KSC LC-39A','value':'KSC LC-39A'},
                                        {'label':'CCAFS SLC-40','value':'CCAFS SLC-40'} 
                                    ],
                                    value='all',
                                    placeholder='Select a Launch Site here',
                                    searchable=True
                                ),
                                
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider',min=0,max=10000,step=1000,
                                    marks={0:'0',2000:'2000',4000:'4000',6000:'6000',
                                           8000:'8000',10000:'10000'},
                                    value=[min_payload,max_payload]),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
# Function decorator to specify function input and output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):
    if entered_site == 'all':
        filtered_df = spacex_df
        fig = px.pie(filtered_df, values='class', 
        names='Launch Site', 
        title='Total Successes by Launch Site')
    else:
        filtered_df = spacex_df[spacex_df['Launch Site']== entered_site]
        fig = px.pie(filtered_df, 
        names='class', 
        title='Total Successes from the Launch Site: ' + str(entered_site))
    
    return(fig)


# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'),
              Input(component_id="payload-slider", component_property="value"))
def get_scatter(entered_site,payload_range):
    if entered_site == 'all':
        filtered_df = spacex_df
        
    else:
        filtered_df = spacex_df[spacex_df['Launch Site']== entered_site]

    
    filtered_df = filtered_df[(filtered_df['Payload Mass (kg)'] > payload_range[0]) 
                          &(filtered_df['Payload Mass (kg)'] < payload_range[1])]
    fig = px.scatter(filtered_df, y="class", x="Payload Mass (kg)", color="Booster Version Short")
    fig.update_xaxes(range=[payload_range[0]-100,payload_range[1]+100])
    fig.update_yaxes(range=[-0.05,1.05])
    return(fig)

# Run the app
if __name__ == '__main__':
    app.run_server()
